
Our dataset consists of clinical data from patients who entered the hospital complaining of chest pain ("angina") during exercise.  The information collected includes:

* `age` : Age of the patient

* `sex` : Sex of the patient

* `cp` : Chest Pain type

    + Value 0: asymptomatic
    + Value 1: typical angina
    + Value 2: atypical angina
    + Value 3: non-anginal pain
   
    
* `trtbps` : resting blood pressure (in mm Hg)

* `chol` : cholesterol in mg/dl fetched via BMI sensor

* `restecg` : resting electrocardiographic results

    + Value 0: normal
    + Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    + Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

* `thalach` : maximum heart rate achieved during exercise

* `output` : the doctor's diagnosis of whether the patient is at risk for a heart attack
    + 0 = not at risk of heart attack
    + 1 = at risk of heart attack

In [23]:
## library imports here

import pandas as pd
import numpy as np 


from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet , LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import r2_score,classification_report, f1_score

from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
ha = pd.read_csv("https://www.dropbox.com/s/aohbr6yb9ifmc8w/heart_attack.csv?dl=1")

## Q1: Natural Multiclass Models

Fit a multiclass KNN, Decision Tree, and LDA for the heart disease data; this time predicting the type of chest pain (categories 0 - 3) that a patient experiences.  For the decision tree, plot the fitted tree, and interpret the first couple splits.


In [3]:
X1 = ha.drop("cp", axis = 1) 
y1 = ha["cp"]

X_train, X_test, y_train, y_test = train_test_split(
    X1, y1, test_size=0.3, random_state=42)

In [4]:
ct = ColumnTransformer(
  [
    ("dummify", 
    OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object)),],
  remainder = "passthrough"
)

pipeKNN = Pipeline(
  [("preprocessing", ct),
  ("KNN", KNeighborsClassifier())]
)

In [5]:
param_grid_knn = {
    "KNN__n_neighbors": range(1,100)}

In [6]:
grid = GridSearchCV(pipeKNN, param_grid_knn, cv=5, scoring='r2')
grid.fit(X1, y1)
print(grid.best_params_)

{'KNN__n_neighbors': 21}


In [7]:
pipeKNN = Pipeline(
  [("preprocessing", ct),
  ("KNN", KNeighborsClassifier(n_neighbors=21))]
)

In [8]:
pipeKNN.fit(X1,y1)

preds = pipeKNN.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.55      0.94      0.69        36
           1       0.00      0.00      0.00        17
           2       0.47      0.38      0.42        24
           3       0.00      0.00      0.00         5

    accuracy                           0.52        82
   macro avg       0.26      0.33      0.28        82
weighted avg       0.38      0.52      0.43        82



C:\Users\ldcal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ldcal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ldcal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
pipeTree = Pipeline(
  [("preprocessing", ct),
  ("Tree", DecisionTreeClassifier())]
)

param_grid_Tree = {
    "Tree__max_depth": [None, 3, 5, 8, 12],
    "Tree__min_samples_split": [2, 5, 10, 20],
    "Tree__min_samples_leaf": [1, 2, 5, 10],
    "Tree__splitter": ["best", "random"],
    "Tree__ccp_alpha": [0.0, 0.0005, 0.001, 0.005]  
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

tree_gs = GridSearchCV(pipeTree, param_grid_Tree, cv=cv, scoring = "f1_macro", n_jobs = -1)

tree_gs.fit(X1, y1)
print(tree_gs.best_params_)

{'Tree__ccp_alpha': 0.0005, 'Tree__max_depth': None, 'Tree__min_samples_leaf': 1, 'Tree__min_samples_split': 10, 'Tree__splitter': 'random'}


In [10]:
pipeTree = Pipeline(
  [("preprocessing", ct),
  ("Tree", DecisionTreeClassifier(ccp_alpha=0.005, max_depth=None, min_samples_leaf=1, min_samples_split=10,
  splitter = "random"))]
)

In [11]:
cross_val_score(pipeTree,X1, y1,cv = cv, scoring = "f1_macro").mean()

np.float64(0.31143143507633464)

In [12]:
pipeTree.fit(X_train,y_train)

Treepreds = pipeTree.predict(X_test)

print(classification_report(y_test, Treepreds))

              precision    recall  f1-score   support

           0       0.62      0.64      0.63        36
           1       0.00      0.00      0.00        17
           2       0.36      0.58      0.44        24
           3       0.00      0.00      0.00         5

    accuracy                           0.45        82
   macro avg       0.25      0.31      0.27        82
weighted avg       0.38      0.45      0.41        82



## Q2:  OvR

Create a new column in the `ha` dataset called `cp_is_3`, which is equal to `1` if the `cp` variable is equal to `3` and `0` otherwise.

Then, fit a Logistic Regression to predict this new target, and report the **F1 Score**.

Repeat for the other three `cp` categories.  Which category was the OvR approach best at distinguishing?

In [17]:
ha["cp_is_3"] = np.where(ha['cp'] == 3, 1, 0)
ha["cp_is_2"] = np.where(ha['cp'] == 2, 1, 0)
ha["cp_is_1"] = np.where(ha['cp'] == 1, 1, 0)
ha["cp_is_0"] = np.where(ha['cp'] == 0, 1, 0)

In [18]:
pipeOvr = Pipeline(
  [("preprocessing", ct),
  ("ovr",OneVsRestClassifier(LogisticRegression(max_iter=5000)))]
  )

In [28]:
X2 = ha.drop(["cp", "cp_is_3", "cp_is_2", "cp_is_1", "cp_is_0"], axis = 1)

In [33]:
rows = []
for k in [0, 1, 2, 3]:
    yk = ha.get(f"cp_is_{k}", (ha["cp"] == k).astype(int)).astype(int)
    scores = cross_val_score(
        estimator=pipeOvr,
        X=X2,
        y=yk,
        cv=cv,
        scoring="f1_macro",
        n_jobs=-1
    )
    rows.append({"cp": k, "F1_mean": scores.mean(), "F1_std": scores.std()})

In [34]:
results = pd.DataFrame(rows).sort_values("F1_mean", ascending=False)

results

,cp,F1_mean,F1_std
0,0,0.733369,0.035798
3,3,0.479921,0.005878
1,1,0.478069,0.050049
2,2,0.455515,0.050772


## Q3: OvO

Reduce your dataset to only the `0` and `1` types of chest pain.

Then, fit a Logistic Regression to predict between the two groups, and report the **ROC-AUC**.  

Repeat comparing category `0` to `2` and `3`.  Which pair was the OvO approach best at distinguishing?

In [52]:
ha_01 = ha[(ha["cp"] == 0) | (ha["cp"] == 1)]

X3_1 = ha_01[["trtbps", "chol", "age"]]
y3_1 = ha_01["cp"]


logistic_model =Pipeline(
    [("scale", StandardScaler()),
    ("model", LogisticRegression())]
)

fit1 = logistic_model.fit(X3_1, y3_1)

In [53]:
cross_val_score(fit1, X3_1, y3_1,
                                cv=5, scoring="roc_auc").mean()

np.float64(0.6016623931623932)

In [54]:
ha_02 = ha[(ha["cp"] == 0) | (ha["cp"] == 2)]

X3_2 = ha_02[["trtbps", "chol", "age"]]
y3_2 = ha_02["cp"]


logistic_model =Pipeline(
    [("scale", StandardScaler()),
    ("model", LogisticRegression())]
)

fit2 = logistic_model.fit(X3_2, y3_2)

In [55]:
cross_val_score(fit2, X3_2, y3_2,
                                cv=5, scoring="roc_auc").mean()

np.float64(0.5593246606334841)

In [56]:
ha_03 = ha[(ha["cp"] == 0) | (ha["cp"] == 3)]

X3_3 = ha_03[["trtbps", "chol", "age"]]
y3_3 = ha_03["cp"]


logistic_model =Pipeline(
    [("scale", StandardScaler()),
    ("model", LogisticRegression())]
)

fit3 = logistic_model.fit(X3_3, y3_3)

In [57]:
cross_val_score(fit3, X3_3, y3_3,
                                cv=5, scoring="roc_auc").mean()

np.float64(0.5585384615384615)

The pair that ovo was best at modeling was cp 0 and 1 likely because it is these values probably have closer values so segmenting them is easier